In [1]:
from plotlib.loaders import *
from plotlib.plotters import *
from phdconf import config 

import pandas as pd
import copy
import subprocess

In [36]:
print(config.DATA_DIR)
# queries = load_queries(config.AUS_TOPIC_PATH)

/home/danlocke/go/src/github.com/dan-locke/phd-data/


In [37]:
queries = load_queries(os.path.join(config.DATA_DIR, 'all-leg-queries.json'))
broad, specific = load_query_types(queries)

paths = ['leg-casename.run', 'leg-citation.run', 'leg-count.run', 'leg-date.run', 'leg-query.run', 'leg-sec-count.run', 'leg-specific-sec-count.run', 'leg-interp.run', 'leg-comb.run', 'auspdfs-unigram_dir_mu_1050.00.run', 'filtered-phrasestop-unigram_dir_mu_1050.00.run']
paths = ['all-leg-queries-'+x for x in paths]
names = ['casenames', 'citations', 'counts', 'date', 'query', 'section', 'specific_section', 'interp', 'comb', 'pdf', 'text']
displays = ['Casename', 'Citation', 'Legislation', 'Date', 'Query', 'Section', 'Specific Sec', 'Interp', 'Combined', 'PDF', 'Cleaned']

In [38]:
# def create_folds(queries, k: int):
#     l = len(queries)
#     q_set = set(queries)
#     inds = [x for x in range(l)]
#     np.random.shuffle(inds)
#     queries = np.array(queries)
#     queries = queries[inds]
    
#     avg = l / float(k)
#     parts = []
#     last = 0.0
    
#     test = []

#     while last < l:
#         test.append(list(queries[int(last):int(last + avg)]))
#         last += avg
     
#     for i in range(len(parts)):
#         train.append()
    
#     train = []
#     for i in range(len(test)):
#         train.append(list(q_set.difference(test[i])))
    
#     return train, test
    
# train, test = create_folds(list(queries.keys()), 5)

In [39]:
# # write folds to file 
# with open('ausleg-folds.txt', 'w') as f:
#     for tr, te in zip(train, test):
#         f.write('{0} {1}\n'.format(tr, te))

In [40]:
# dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(config.BASE_DIR, 'dirichlet_prior'), paths)
dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '/home/danlocke/phd-generated/legislative/', paths, per_query=True)

In [41]:
print(dfs[0])

     recip_rank  unjudged@20  recall_20  recall_100    ndcg   set_P  \
1        1.0000         13.0     0.2857      1.0000  0.6559  0.1707   
10       0.0020         19.0     0.0000      0.0000  0.1252  0.0003   
103      1.0000         18.0     0.2000      0.2000  0.3296  0.0097   
104      1.0000         16.0     0.0870      0.5217  0.5179  0.1111   
106      0.0021         15.0     0.0000      0.0000  0.1746  0.0013   
119      0.5000          0.0     1.0000      1.0000  0.7168  0.5000   
14       1.0000          0.0     1.0000      1.0000  1.0000  1.0000   
16       0.1250          0.0     0.5000      0.5000  0.1199  0.0769   
17       0.0172         18.0     0.0000      0.3333  0.1235  0.0068   
18       0.0000          0.0     0.0000      0.0000  0.0000  0.0000   
2        0.0312         19.0     0.0000      0.6667  0.2157  0.0345   
20       1.0000          0.0     1.0000      1.0000  0.7626  0.5000   
26       1.0000          0.0     1.0000      1.0000  0.7769  0.6875   
30    

In [92]:
def calc_sig(dfs, comps, names, metrics):
    d = pd.DataFrame(columns=dfs[0].columns)
    all_ind = set()
    for df in dfs:
        all_ind.update(df.index)
        
    for i in range(len(dfs)): 
        ind = all_ind - set(dfs[i].index)
        ncol = len(dfs[i].columns)
        for x in ind: 
            dfs[i].loc[x] = [0.0] * ncol
        dfs[i].sort_index(inplace=True)
    
    symbols = ['*', '\dagger', '\cdot', '\textdollar', '\textyen', '\textcent', '\texteuro', '\text', '\cdot', '\circ', '\centerdot', '\diamond']
    comp_res = []
    for c in comps: 
        res = {}
        for m in metrics:
            res[m] = dfs[c][m]
        comp_res.append((c, res))

    for i, df in enumerate(dfs):
        av = df.mean()
        for j, res in enumerate(comp_res):
            if i <= res[0]: 
                continue
            for m in metrics:
#                 print(res[m])
#                 print(df[m])
                p = stats.ttest_rel(res[1][m], df[m]).pvalue
                if p < 0.05: 
                    sym = symbols[j]
                    if p < 0.01:
                        sym = sym*2
                    if isinstance(av[m], str):
                        av[m] = av[m][:-2] + sym + "}$"
                    else:
                        av[m] = "{0:.4f}$^{{".format(av[m]) + sym + "}$"
                elif isinstance(av[m], float):
                    av[m] = "{0:.4f}$^{{}}$".format(av[m])

        d.loc[names[i]] = av
    print(d.round(4).filter(metrics, axis='columns').rename(metrics, axis='columns').to_latex(escape=False))
    return d.filter(metrics, axis='columns').rename(metrics, axis='columns')
                    

metrics = {}
metrics['set_P'] = 'P'
metrics['set_recall'] = 'R'
metrics['set_F_.5'] = 'F_{0.5}'
all_leg_df = calc_sig(dfs, [4, 9, 10], displays, metrics)  

\begin{tabular}{llll}
\toprule
{} &                           P &                            R &                     F_{0.5} \\
\midrule
Casename     &                    0.135281 &                     0.739455 &                     0.16535 \\
Citation     &                    0.135281 &                     0.739455 &                     0.16535 \\
Legislation  &                    0.135281 &                     0.739455 &                     0.16535 \\
Date         &                    0.135281 &                     0.739455 &                     0.16535 \\
Query        &                    0.135281 &                     0.739455 &                     0.16535 \\
Section      &                 0.1353$^{}$ &                  0.7395$^{}$ &                 0.1653$^{}$ \\
Specific Sec &                 0.1353$^{}$ &                  0.7395$^{}$ &                 0.1653$^{}$ \\
Interp       &                 0.1353$^{}$ &                  0.7395$^{}$ &                 0.1653$^{}$ \\
Combine

In [94]:
stats.ttest_rel(dfs[9]['recip_rank'], dfs[10]['recip_rank'])

Ttest_relResult(statistic=2.592600020740532, pvalue=0.013142732858364743)

In [43]:
sub_df = all_leg_df.reindex(['Query', 'PDF', 'Cleaned'])
sub_df = sub_df.rename(index={'PDF': '\texttt{PDF}', 'Cleaned': '\texttt{cleaned}', 'Query': '\texttt{bool}'})
write_table('tables/ausl-leg-bool', bold_max(sub_df).to_latex(escape=False))

In [44]:
metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']
calc_sig(dfs[:-2], [4], displays[:-2], metrics)  

\begin{tabular}{lllllll}
\toprule
{} &             RR &         ERR@20 &           R@20 &          R@100 &           NDCG &            RBP \\
\midrule
Casename     &       0.320271 &       0.197055 &       0.257776 &        0.48086 &        0.34169 &       0.147445 \\
Citation     &       0.238398 &       0.135637 &       0.275395 &       0.480848 &       0.321067 &       0.114488 \\
Legislation  &         0.2219 &       0.125844 &        0.22249 &       0.431017 &       0.299038 &       0.099986 \\
Date         &       0.273671 &       0.157686 &       0.253379 &       0.482286 &       0.322574 &        0.12661 \\
Query        &       0.577874 &       0.327866 &       0.426814 &       0.584733 &       0.449486 &       0.311152 \\
Section      &  0.1581$^{**}$ &  0.0664$^{**}$ &  0.1894$^{**}$ &  0.4392$^{**}$ &  0.2723$^{**}$ &  0.0596$^{**}$ \\
Specific Sec &  0.2742$^{**}$ &  0.1612$^{**}$ &  0.2565$^{**}$ &  0.4842$^{**}$ &  0.3217$^{**}$ &  0.1055$^{**}$ \\
Interp       &   0.4794

,RR,ERR@20,R@20,R@100,NDCG,RBP
Casename,0.320271,0.197055,0.257776,0.48086,0.34169,0.147445
Citation,0.238398,0.135637,0.275395,0.480848,0.321067,0.114488
Legislation,0.2219,0.125844,0.22249,0.431017,0.299038,0.099986
Date,0.273671,0.157686,0.253379,0.482286,0.322574,0.12661
Query,0.577874,0.327866,0.426814,0.584733,0.449486,0.311152
Section,0.1581$^{**}$,0.0664$^{**}$,0.1894$^{**}$,0.4392$^{**}$,0.2723$^{**}$,0.0596$^{**}$
Specific Sec,0.2742$^{**}$,0.1612$^{**}$,0.2565$^{**}$,0.4842$^{**}$,0.3217$^{**}$,0.1055$^{**}$
Interp,0.4794$^{*}$,0.2782$^{*}$,0.3821$^{*}$,0.5515$^{*}$,0.4124$^{**}$,0.2538$^{**}$
Combined,0.5526$^{}$,0.3158$^{}$,0.4300$^{}$,0.5800$^{}$,0.4449$^{}$,0.3050$^{**}$


In [45]:
paths = ['leg-casename.run', 'leg-citation.run', 'leg-count.run', 'leg-date.run', 'leg-query.run', 'leg-sec-count.run', 'leg-specific-sec-count.run', 'leg-interp.run', 'leg-comb.run', 'auspdfs-unigram_dir_mu_1050.00.run', 'filtered-phrasestop-unigram_dir_mu_1050.00.run']
paths = ['all-leg-queries-'+x for x in paths]
names = ['casenames', 'citations', 'counts', 'date', 'query', 'section', 'specific_section', 'interp', 'comb', 'pdf', 'text']
    
# names[4]
inter = Interpolater(os.path.join('/home/danlocke/phd-generated/legislative/', paths[4]), normalize=True)

dfs = []
for d in paths[:4]+paths[5:-4]: 
    print(d)
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join('/home/danlocke/phd-generated/legislative/', d), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
    dfs.append(interped_dfs)

all-leg-queries-leg-casename.run
all-leg-queries-leg-citation.run
all-leg-queries-leg-count.run
all-leg-queries-leg-date.run
all-leg-queries-leg-sec-count.run
all-leg-queries-leg-specific-sec-count.run


In [12]:
# inter.interpolate(os.path.join('/home/danlocke/phd-generated/legislative/', paths[3]), 0.5, 'tmp.run')
# load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0]

In [46]:
tt_folds = read_folds('ausleg-folds.txt')

In [47]:
base_query = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', [os.path.join('/home/danlocke/phd-generated/legislative/', paths[4])], per_query=True)[0]

In [48]:
metrics = copy.copy(config.METRIC_NAMES)

ntlm_df = pd.DataFrame(columns=metrics)

for ab, runs in zip(names[2:4]+names[5:-4], dfs[2:]):
    print(ab)
    ntlm_cross = cross_validation(runs, tt_folds, metrics, base_query)
    ntlm_df.loc[ab] = ntlm_cross[0]

counts
date
section
specific_section


In [49]:
ntlm_df.loc['$R$'] = base_query.mean().round(4)
ntlm_df = ntlm_df.reindex(['$R$', 'counts', 'section', 'specific_section', 'date'])
print(ntlm_df.rename(columns=metrics).drop(['Unjudged@20', 'R@100'],axis='columns').to_latex(escape=False))

\begin{tabular}{llllll}
\toprule
{} &      RR &  ERR@20 &          R@20 &           NDCG &           RBP \\
\midrule
$R$              &  0.5779 &  0.3279 &        0.4268 &         0.4495 &        0.3112 \\
counts           &  0.5254 &  0.2970 &  0.3732$^{*}$ &  0.3573$^{**}$ &  0.2807$^{*}$ \\
section          &  0.5148 &  0.3020 &  0.3732$^{*}$ &  0.3568$^{**}$ &  0.2826$^{*}$ \\
specific_section &  0.5412 &  0.3143 &  0.3732$^{*}$ &  0.3620$^{**}$ &  0.2825$^{*}$ \\
date             &  0.5424 &  0.3179 &  0.3661$^{*}$ &  0.3608$^{**}$ &  0.2809$^{*}$ \\
\bottomrule
\end{tabular}



In [50]:
ausl_leg_df = ntlm_df.reindex(['$R$', 'counts', 'section'])
ausl_leg_df = ausl_leg_df.rename(index={'counts': 'leg', 'section': 'sec'})
write_table('tables/ausl-leg-ranking', bold_max(ausl_leg_df).rename(columns=metrics).drop(['Unjudged@20', 'R@100'],axis='columns').to_latex(escape=False))

In [51]:
ausl_date_df = ntlm_df.reindex(['$R$', 'date'])
write_table('tables/ausl-date-ranking', bold_max(ausl_date_df).rename(columns=metrics).drop(['Unjudged@20', 'R@100'],axis='columns').to_latex(escape=False))

In [19]:
def get_missed(run_name: str):    
    retrieved = {}
    with open('/home/danlocke/phd-generated/legislative/{0}.run'.format(run_name)) as f:
        for line in f: 
            parts = line.strip().split()
            if parts[0] not in retrieved:
                retrieved[parts[0]] = set()
            retrieved[parts[0]].add(parts[2])
            
    relevant = {}
    with open(config.AUS_QREL_PATH) as f:
        for line in f:
            parts = line.strip().split()
            if parts[0] not in retrieved:
                continue
            if parts[0] not in relevant:
                relevant[parts[0]] = set()
            if parts[3] == "1":
                relevant[parts[0]].add(parts[2])
                
                
    for topic in relevant.keys():
        items = copy.copy(relevant[topic])
        for doc in items:
            if doc in retrieved[topic]: 
                relevant[topic].remove(doc)
    
    return(relevant)
    
missed = get_missed('all-leg-queries-leg-query')


# not cite - 33
# 2 - 2006QCA155, 2003FCA50
# 17 - 2014FCA765, 
# 35 - 2003FCAFC109, 2014QSC281
# 47 - 2005FCA494, 2008FCAFC170
# 55 - 2009FCA1502, 2013FCA1038, 2011FCA1159, 2013QCA104, 2015FCA741
# 60 - 2015QSC149, 2014QCA183
# 61 - 2013QCA161, 2016FCA243, 2002QCA225, 2012QCA100, 2009QSC417, 2016QCA215, 2010QSC122, 2008FCA0865, 2014FCA585
# 66 - 2006QSC349
# 77 - 2005QCA277, 2010FCA769, 2005FCA918, 2010FCA285, 2018FCA33
# 78 - 2010FCA285, 2005FCA918, 2018FCA33
# 86 - 2011QDC213

# reason for this is might be appeal or costs decision

# cite prev leg - 4
# 36 - 2001QSC211
# 38 - 2004FCA5
# 92 - 2011QSC307, 2009QCA75

# cite other sec from same act or another act - 15
# 17 - 2011FCA1063
# 55 - 2019QSC144
# 60 - 2001QSC444, 2007QCA185, 2007QSC92, 2008QSC4, 2011QSC88, 2012QDC73, 2006QSC274, 2004QSC272, 2009QSC192, 
# 78 - 2018QCA79, 2007QSC262
# 96 - 2018FCA657, 2018FCA1405

In [20]:
# missed_standard = get_missed('all-leg-queries-leg-linked-rep')

In [21]:
# a = missed_standard
# b = missed
# for k, v in a.items():
#     if k in b:
#         for v1 in v:
#             if v1 not in b[k]:
#                 print(k, v1)

In [22]:
# for k, v in missed_standard.items():
#     print(k, v)

In [52]:
# for k, v in missed.items():
#     print(k, v)
dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '/home/danlocke/phd-generated/legislative/', paths, per_query=True)

In [53]:
topics = ['legislative-queries', 'case-topics']
expansion_paths = ['leg-exp.run', 'leg-filtered-exp.run', 'leg-linked-all.run', 'leg-linked-rep.run', 'leg-linked-same.run']
expansion_paths = ['all-leg-queries-'+x for x in expansion_paths]

results = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '/home/danlocke/phd-generated/legislative/', expansion_paths, per_query=True)

In [54]:
len(dfs[-3:]+results)

8

In [55]:
metrics['set_P'] = 'P'
metrics['set_recall'] = 'R'
metrics['set_F_.5'] = 'F_{0.5}'
leg_filter_df = calc_sig(dfs[-3:]+results, [0, 1, 2], displays[-3:]+['leg-exp', 'leg-filtered-exp', 'leg-linked-all', 'leg-linked-rep', 'leg-linked-same'], metrics)    

\begin{tabular}{lllllllllll}
\toprule
{} &                                     RR &                                 ERR@20 &                                   R@20 &                                  R@100 &                                   NDCG &                                    RBP &                             Unjudged@20 &                                      P &                                      R &                                F_{0.5} \\
\midrule
Combined         &                               0.552605 &                               0.315797 &                               0.430007 &                               0.579971 &                               0.444857 &                               0.304998 &                                8.071429 &                               0.135281 &                               0.739455 &                                0.16535 \\
PDF              &                            0.5410$^{}$ &                            0.3021$^{}$ &     

In [70]:
leg_filter_df = leg_filter_df.rename(index={'PDF': '\texttt{PDF}', 'Cleaned': '\texttt{cleaned}', 'Combined': '\texttt{bool}', 
                                        'leg-linked-all': 'all-link', 'leg-linked-rep': 'rep-link', 'leg-linked-same': 'same-link'})
leg_filter_df = leg_filter_df.applymap(lambda x: round(x, 4) 
    if isinstance(x, (int, float)) else x)
write_table('tables/ausl-leg-exp', bold_max(leg_filter_df.round(4)).rename(columns=metrics).to_latex(escape=False))

In [71]:
ndf = leg_filter_df.reindex(['\texttt{bool}', '\texttt{cleaned}', '\texttt{PDF}', 'leg-exp'])
ndf = ndf.applymap(lambda x: round(x, 4) if isinstance(x, (int, float)) else x)
write_table('tables/ausl-leg-term-exp', bold_max(ndf).rename(columns=metrics).to_latex(escape=False))

In [29]:
metrics = copy.copy(config.METRIC_NAMES)
del metrics ['unjudged@20']
calc_sig(dfs+results, [4, 9, 10], displays+['leg-exp', 'leg-filtered-exp', 'leg-linked-all', 'leg-linked-rep', 'leg-linked-same'], metrics)    

\begin{tabular}{lllllll}
\toprule
{} &                                     RR &                                 ERR@20 &                                   R@20 &                                  R@100 &                                   NDCG &                                    RBP \\
\midrule
Casename         &              0.3203$^{\dagger\dagger}$ &                     0.1971$^{\dagger}$ &              0.2578$^{\dagger\dagger}$ &                     0.4809$^{\dagger}$ &                            0.3417$^{}$ &              0.1474$^{\dagger\dagger}$ \\
Citation         &                     0.2384$^{**\cdot}$ &                          0.1356$^{**}$ &                          0.2754$^{**}$ &                          0.4808$^{**}$ &                          0.3211$^{**}$ &                          0.1145$^{**}$ \\
Legislation      &            0.2219$^{**\dagger\dagger}$ &            0.1258$^{**\dagger\dagger}$ &            0.2225$^{**\dagger\dagger}$ &            0.4310$^{**\dagger\d

,RR,ERR@20,R@20,R@100,NDCG,RBP
Casename,0.3203$^{\dagger\dagger}$,0.1971$^{\dagger}$,0.2578$^{\dagger\dagger}$,0.4809$^{\dagger}$,0.3417$^{}$,0.1474$^{\dagger\dagger}$
Citation,0.2384$^{**\cdot}$,0.1356$^{**}$,0.2754$^{**}$,0.4808$^{**}$,0.3211$^{**}$,0.1145$^{**}$
Legislation,0.2219$^{**\dagger\dagger}$,0.1258$^{**\dagger\dagger}$,0.2225$^{**\dagger\dagger}$,0.4310$^{**\dagger\dagger}$,0.2990$^{**\dagger\dagger}$,0.1000$^{**\dagger\dagger}$
Date,0.2737$^{**\dagger\dagger}$,0.1577$^{**\dagger\dagger}$,0.2534$^{**\dagger\dagger}$,0.4823$^{**\dagger}$,0.3226$^{**\dagger}$,0.1266$^{**\dagger\dagger}$
Query,0.5779$^{\cdot\cdot}$,0.3279$^{\cdot\cdot}$,0.4268$^{\cdot\cdot}$,0.5847$^{}$,0.4495$^{\cdot\cdot}$,0.3112$^{\cdot\cdot}$
Section,0.1581$^{**\dagger\dagger\cdot\cdot}$,0.0664$^{**\dagger\dagger\cdot}$,0.1894$^{**\dagger\dagger}$,0.4392$^{**\dagger\dagger}$,0.2723$^{**\dagger\dagger}$,0.0596$^{**\dagger\dagger\cdot\cdot}$
Specific Sec,0.2742$^{**\dagger\dagger}$,0.1612$^{**\dagger\dagger}$,0.2565$^{**\dagger\dagger}$,0.4842$^{**\dagger}$,0.3217$^{**\dagger}$,0.1055$^{**\dagger\dagger\cdot}$
Interp,0.4794$^{*}$,0.2782$^{*}$,0.3821$^{*\cdot}$,0.5515$^{*}$,0.4124$^{**\cdot\cdot}$,0.2538$^{**}$
Combined,0.5526$^{\cdot}$,0.3158$^{\cdot\cdot}$,0.4300$^{\cdot\cdot}$,0.5800$^{}$,0.4449$^{\cdot\cdot}$,0.3050$^{**\cdot\cdot}$
PDF,0.5410$^{\cdot}$,0.3021$^{\cdot}$,0.4044$^{\cdot\cdot}$,0.6409$^{\cdot\cdot}$,0.4160$^{\cdot\cdot}$,0.2954$^{\cdot\cdot}$


In [30]:
results[-1][(results[-1]-dfs[-3])['set_recall'] < 0]

,recip_rank,unjudged@20,recall_20,recall_100,ndcg,set_P,set_recall,set_F_.5,rbp@0.10,rbp-res@0.10,rbp@0.50,rbp-res@0.50,rbp@0.80,rbp-res@0.80,err@20


In [31]:
dfs[-3].loc[38]

recip_rank      0.3333
unjudged@20     0.0000
recall_20       0.2500
recall_100      0.2500
ndcg            0.1757
set_P           0.2500
set_recall      0.2500
set_F_.5        0.2500
rbp@0.10        0.0090
rbp-res@0.10    0.0001
rbp@0.50        0.1250
rbp-res@0.50    0.0625
rbp@0.80        0.1280
rbp-res@0.80    0.4096
err@20          0.1250
Name: 38, dtype: float64

In [32]:
miss_b = get_missed('legislative-queries-leg-linked-all') 
miss_a = get_missed('legislative-queries-leg-exp') 
# first is missing from baseline 
# for k, v in miss_a.items():
#     if len(v) > 0 or len(miss_b[k]) > 0:
#         print(k, v, '|', miss_b[k])
# print('-'*20)
# miss_b = get_missed('legislative-queries-leg-linked-same') 
# miss_a = get_missed('legislative-queries-leg-exp') 
# for k, v in miss_a.items():
#     if len(v) > 0 or len(miss_b[k]) > 0:
#         print(k, v, '|', miss_b[k])
# print('-'*20)
# miss_b = get_missed('legislative-queries-leg-linked-rep') 
# miss_a = get_missed('legislative-queries-leg-exp') 
# for k, v in miss_a.items():
#     if len(v) > 0 or len(miss_b[k]) > 0:
#         print(k, v, '|', miss_b[k])
# print('-'*20)

FileNotFoundError: [Errno 2] No such file or directory: '/home/danlocke/phd-generated/legislative/legislative-queries-leg-linked-all.run'

In [33]:
get_missed('case-topics-leg-exp') 

FileNotFoundError: [Errno 2] No such file or directory: '/home/danlocke/phd-generated/legislative/case-topics-leg-exp.run'

In [34]:
runs = [os.path.join('/home/danlocke/phd-generated/legislative/', x) for x in ['leg-casename.run', 'leg-citation.run', 'leg-count.run', 'leg-query.run', 'leg-sec-count.run', 'leg-specific-sec-count.run', 'leg-interp.run']]
comb_method = ['combmax', 'combanz', 'combmed', 'combmin', 'combmax', 'combmnz', 'combsum']
for method in comb_method:
    f = open("{0}-tmp.run".format(method), "w")
    subprocess.call(['polyfuse', method, '-n', 'minsum'] + runs, stdout=f)
    f.close()

In [35]:
comb_method_dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '.', ['{0}-tmp.run'.format(x) for x in comb_method])
print(pd.DataFrame.from_dict({k: comb_method_dfs[i] for i, k in enumerate(comb_method)}))

CalledProcessError: Command '['trec_eval', '-q', '-m', 'recall.20,100', '-m', 'ndcg', '-m', 'recip_rank', '-m', 'set_P', '-m', 'set_recall', '-m', 'relstring.20', '-m', 'set_F..5', '-l', '1', '/home/danlocke/go/src/github.com/dan-locke/phd-data/aus/filtered-qrels.txt', './combmax-tmp.run']' returned non-zero exit status 2.

In [36]:
norm_type = ['minmax', 'std', 'sum', 'minsum']
for nt in norm_type: 
    f = open("{0}-tmp.run".format(nt), "w")
    subprocess.call(['polyfuse', 'combmax', '-n', nt] + runs, stdout=f)
    f.close()
    
norm_type_dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '.', ['{0}-tmp.run'.format(x) for x in norm_type])
print(pd.DataFrame.from_dict({k: norm_type_dfs[i] for i, k in enumerate(norm_type)}))

CalledProcessError: Command '['trec_eval', '-q', '-m', 'recall.20,100', '-m', 'ndcg', '-m', 'recip_rank', '-m', 'set_P', '-m', 'set_recall', '-m', 'relstring.20', '-m', 'set_F..5', '-l', '1', '/home/danlocke/go/src/github.com/dan-locke/phd-data/aus/filtered-qrels.txt', './minmax-tmp.run']' returned non-zero exit status 2.

In [12]:
def get_num_results(path: str):
    res = {}
    with open(path) as f:
        for line in f:
            topic = line.split()[0]
            v = res.get(topic, 0)
            res[topic] = v + 1
                
    return res

In [13]:
# get_num_results(os.path.join('/home/danlocke/phd-generated/legislative/', 'case-topics-leg-exp.run'))

{'1': 81,
 '2': 115,
 '9': 6046,
 '10': 6046,
 '17': 145,
 '35': 39,
 '47': 1593,
 '54': 6046,
 '55': 96,
 '60': 95,
 '61': 102,
 '77': 692,
 '78': 692,
 '86': 333,
 '87': 333,
 '88': 457,
 '89': 457,
 '96': 91,
 '97': 31,
 '103': 205,
 '104': 205}

In [62]:
# get_num_results(os.path.join('/home/danlocke/phd-generated/legislative/', 'legislative-queries-flattened-unigram_dir_mu_2400.00.run'))

# clear files

{'14': 31135,
 '16': 46807,
 '18': 34744,
 '20': 11552,
 '26': 29603,
 '30': 32583,
 '31': 36319,
 '36': 12118,
 '38': 41085,
 '40': 37361,
 '42': 23058,
 '48': 31855,
 '51': 28337,
 '56': 16271,
 '63': 49538,
 '66': 35510,
 '72': 19829,
 '79': 39939,
 '92': 39840,
 '106': 24429,
 '119': 39774,
 '120': 44638,
 '121': 47314,
 '122': 50168,
 '123': 43030,
 '124': 47684,
 '125': 52388,
 '127': 49601,
 '128': 32738,
 '129': 52254,
 '130': 34914,
 '131': 50825,
 '133': 51241,
 '134': 3901,
 '135': 42452,
 '136': 31339,
 '137': 51718,
 '138': 51382,
 '139': 43415}

In [37]:
def load_prf(path: str): 
    with open(path) as f:
        data = json.load(f)
        return data
        
prf_data = load_prf('/home/danlocke/go/src/github.com/dan-locke/phd/leg-prf/leg-prf.json')

In [38]:
secs = []
legs = []
for i in range(0, 4):
    secs.append([])
    legs.append([])

for section in prf_data['sections']:
    q_leg = set()
    q_sec = []
    
    if section in prf_data['Found']:
        q_leg = set(prf_data['Found'][section][0])
        if prf_data['Found'][section][1] is not None:
            q_sec = set(prf_data['Found'][section][1])
    
    for i in range(0, 4):
        if prf_data['legislation'][section][i] is None:
            continue
            
        for l in prf_data['legislation'][section][i].items():
            if l[0] == 'act' or l[0] == 'acts':
                continue
            legs[i].append((section, l[0], l[1], l[0] in q_leg))
            
        for s in prf_data['sections'][section][i].items():
            secs[i].append((section, s[0], s[1], s[0] in q_sec))

#         legs = sorted(prf_data['legislation'][section][i].items(), key=lambda x: x[1], reverse=True)
#         secs = sorted(prf_data['sections'][section][i].items(), key=lambda x: x[1], reverse=True)
                    
    
#     break

In [39]:
print(prf_data['Found'].keys())
print(set(prf_data['sections'].keys()).difference(set(prf_data['Found'].keys())))
print(len(prf_data['sections'].keys()))
print(prf_data['legislation'].keys())

dict_keys(['1', '10', '103', '104', '106', '119', '14', '16', '17', '18', '2', '20', '26', '30', '31', '35', '36', '38', '40', '42', '47', '48', '51', '54', '55', '56', '60', '61', '63', '66', '77', '78', '79', '86', '87', '88', '89', '9', '92', '96', '97'])
{'72'}
42
dict_keys(['1', '10', '103', '104', '106', '119', '14', '16', '17', '18', '2', '20', '26', '30', '31', '35', '36', '38', '40', '42', '47', '48', '51', '54', '55', '56', '60', '61', '63', '66', '72', '77', '78', '79', '86', '87', '88', '89', '9', '92', '96', '97'])


In [40]:
sec_df = pd.DataFrame(data=secs[0], columns=['query', 'item', 'count', 'in_query'])
f, ax = plt.subplots(1)
sec_scatter = sns.scatterplot(x='count', y='query',
              hue='in_query',
              data=sec_df)

f.set_size_inches(16, 4)
ax.set_xlabel('Frequency', fontsize=15)
ax.axes.yaxis.set_visible(False)
ax.axes.tick_params(labelsize=12)
ax.axes.yaxis.set_visible(False)
ax.get_legend().remove()
f.savefig('figures/aus-sec-prf.pdf')

<Figure size 1152x288 with 1 Axes>

In [41]:
leg_df = pd.DataFrame(data=legs[0], columns=['query', 'item', 'count', 'in_query'])
f, ax = plt.subplots(1)
leg_scatter = sns.scatterplot(x='count', y='query',
              hue='in_query',
              data=leg_df, ax=ax)

f.set_size_inches(16, 4)
# ax.axes.xaxis.set_visible(False)
ax.set_xlabel('Frequency', fontsize=15)
ax.axes.yaxis.set_visible(False)
ax.axes.tick_params(labelsize=12)
ax.get_legend().remove()
f.savefig('figures/aus-leg-prf.pdf')

<Figure size 1152x288 with 1 Axes>